In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d ananthu017/emotion-detection-fer

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D

In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/emotion-detection-fer.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/emotion-detection-fer.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_data_dir='/content/train'
test_data_dir='/content/train'

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation_datagen=ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
)

In [ ]:
validation_generator=validation_datagen.flow_from_directory(
    test_data_dir,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
class_labels=['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']
img,label=train_generator.__next__()

In [ ]:
import tensorflow as tf
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
model= tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(512,(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(512,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))
model.compile(
    optimizer = Adam(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
  )

In [ ]:
model.summary()

In [ ]:
train_path="/content/train"
test_path="/content/test"

In [ ]:
import os


In [ ]:
num_train_imgs=0
for root,dirs,files in os.walk(train_path):
  num_train_imgs +=len(files)

In [ ]:
num_train_imgs

In [ ]:
num_test_imgs=0
for root,dirs,files in os.walk(test_path):
  num_test_imgs +=len(files)

In [ ]:
num_test_imgs

In [ ]:
history=model.fit(train_generator,
                  steps_per_epoch=num_train_imgs//32,
                  epochs=13,
                  validation_data=validation_generator,
                  validation_steps=num_test_imgs//32)


In [ ]:
model.save('/content/sample_data/my_model1.h5')
